In [28]:
import nest_asyncio
nest_asyncio.apply()

In [29]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROOQ_API_KEY"] = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama-3.1-8b-instant",temperature=0.7)

result=llm.invoke("Howdy")
result

AIMessage(content="Howdy! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 37, 'total_tokens': 63, 'completion_time': 0.02782042, 'prompt_time': 0.002072599, 'queue_time': 0.215737878, 'total_time': 0.029893019}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_90c2e79dab', 'finish_reason': 'stop', 'logprobs': None}, id='run--2cb23679-c041-49e7-a786-ff42640b15d7-0', usage_metadata={'input_tokens': 37, 'output_tokens': 26, 'total_tokens': 63})

In [30]:
from typing import List
from pydantic import BaseModel,Field


class Section(BaseModel):
    name:str=Field(description="Name for this section of the blog")
    description:str=Field(description="Brief Overview of the main topics and concepts of the section")
    
class Sections(BaseModel):
    sections:List[Section]=Field(description="Sections of the blog")

planner=llm.with_structured_output(Sections)    

In [31]:
from typing import Annotated
from typing_extensions import TypedDict
import operator

class State(TypedDict):
    topic : str
    sections: list[Section]
    completed_sections: Annotated[list,operator.add]
    final_blog:str
    
class WorkerState(TypedDict):
    section: Section
    completed_sections: Annotated[list,operator.add]

In [32]:
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.constants import Send

def orchestrator(state:State):
    """ Orchestrator that genrates a plan for the blog"""    
    blog_sections=planner.invoke(
        [SystemMessage(content="Generate a plan for the blog."),
        HumanMessage(content=f"Here is the blog topic: {state['topic']}")]
        )
    print("Blog sections:", blog_sections)
    return {"sections": blog_sections.sections}

def llm_call(state:State):
    """ Worker writes a section for the blog"""    
    section=llm.invoke([
        SystemMessage(content="Write a blog section following the provided name and description. Include no preamble for each section. Use markdown formatting."),
        HumanMessage(content=f"Here is the section name: {state['section'].name } and description {state['section'].description}")
    ])
    return {"completed_sections": [section.content]}

def assign_worker(state:State):
    """ Assign worker to a section"""    
    return [Send("llm_call", {"section":s}) for s in state["sections"]]

def synthesize_workers(state:State):
    """ Synthesize workers"""  
    completed_sections=state["completed_sections"]
    
    completed_blog_sections="\n\n---\n\n".join(completed_sections)  
    return {"final_blog": completed_blog_sections}


In [35]:
from IPython.display import display,Image
from langgraph.graph import StateGraph,START,END

graph= StateGraph(State)

#Add nodes
graph.add_node("orchestrator",orchestrator)
graph.add_node("llm_call",llm_call)
graph.add_node("synthesize_workers",synthesize_workers)

#Add edges
graph.add_edge(START,"orchestrator")
graph.add_conditional_edges(
    "orchestrator",
    assign_worker,
    ["llm_call"]
)
graph.add_edge("llm_call","synthesize_workers")
graph.add_edge("synthesize_workers",END)

#Compile graph
graph=graph.compile()

graph_image=graph.get_graph().draw_mermaid_png()
display(Image(graph_image))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [ ]:
from IPython.display import Markdown
state=graph.invoke({"topic":"Create a blog on  Agentic AI RAGs"})

Markdown(state["final_blog"])


Blog sections: sections=[Section(name='Introduction to Agentic AI', description='Overview of what agentic AI is, its key features, and importance in modern technology'), Section(name='What are RAGs in AI?', description='Definition, types, and applications of Reasoning, Acting, and Getting (RAGs) in AI systems'), Section(name='Benefits of Agentic AI RAGs', description='How agentic AI RAGs can improve decision-making, problem-solving, and efficiency in various industries'), Section(name='Applications of Agentic AI RAGs', description='Real-world use cases of agentic AI RAGs in areas like customer service, healthcare, finance, and more'), Section(name='Challenges and Limitations of Agentic AI RAGs', description='Potential risks, biases, and limitations of agentic AI RAGs and how to address them'), Section(name='Future of Agentic AI RAGs', description='Emerging trends, advancements, and potential future applications of agentic AI RAGs')]


### Introduction to Agentic AI

#### What is Agentic AI?

Agentic AI refers to a type of artificial intelligence (AI) that exhibits autonomous, self-directed behavior, and decision-making capabilities. This approach to AI development focuses on creating systems that can act independently, make decisions, and take actions in complex, dynamic environments.

#### Key Features of Agentic AI

*   **Autonomy**: Agentic AI systems have the ability to make decisions and act without direct human intervention or supervision.
*   **Self-awareness**: These systems have a sense of their own existence, goals, and limitations, allowing them to adapt and learn from their experiences.
*   **Goal-oriented behavior**: Agentic AI systems are designed to achieve specific objectives, often with a sense of purpose and motivation.
*   **Complex decision-making**: These systems can make decisions based on multiple factors, using reasoning, problem-solving, and learning mechanisms.
*   **Interactions with humans and environments**: Agentic AI systems can interact with humans, other agents, and their environment in a way that is natural and intuitive.

#### Importance of Agentic AI in Modern Technology

Agentic AI has numerous applications in modern technology, including:

*   **Robotics and autonomous systems**: Agentic AI enables the development of robots and autonomous vehicles that can navigate complex environments and make decisions in real-time.
*   **Virtual assistants and chatbots**: Agentic AI-powered virtual assistants and chatbots can provide personalized support, answer questions, and complete tasks for users.
*   **Cybersecurity**: Agentic AI can be used to develop intelligent security systems that can detect and respond to threats in real-time.
*   **Healthcare**: Agentic AI can help diagnose diseases, develop personalized treatment plans, and provide patient care.
*   **Finance**: Agentic AI can be used to develop predictive models, detect anomalies, and make informed investment decisions.

The emergence of agentic AI has the potential to revolutionize various industries and aspects of our lives. However, it also raises important questions about accountability, ethics, and the potential impact on human jobs and relationships.

---

### What are RAGs in AI?

#### Definition

RAGs, which stand for Reasoning, Acting, and Getting, are a framework used in Artificial Intelligence (AI) to describe a basic structure for decision-making and problem-solving in autonomous systems. This framework is typically applied in areas like robotics, autonomous vehicles, and decision support systems.

#### Types of RAGs

There are several types of RAGs, each serving a specific purpose within AI systems:

* **Perception-Action RAGs**: These RAGs are used in situations where the system must sense the environment (perception), reason about it, and then take action based on that reasoning.
* **Planning-Acting RAGs**: This type of RAG is used when the system must create a plan to achieve a goal, and then act on that plan.
* **Learning-Acting RAGs**: These RAGs are applied when the system learns from experience and then uses that knowledge to make decisions and take actions.

#### Applications of RAGs

RAGs have a wide range of applications in AI, including:

* **Robotics**: RAGs are used in robotic systems to enable them to perceive their environment, reason about it, and take actions to achieve a goal.
* **Autonomous Vehicles**: RAGs are applied in self-driving cars to make decisions about navigation, traffic rules, and obstacle avoidance.
* **Decision Support Systems**: RAGs are used in decision support systems to analyze data, reason about it, and provide recommendations to users.
* **Game Playing**: RAGs are applied in game playing systems to analyze the game state, reason about possible moves, and make decisions about which move to make.

#### Benefits of RAGs

The use of RAGs in AI systems provides several benefits, including:

* **Improved Decision-Making**: RAGs enable AI systems to make more informed decisions based on their reasoning and perception of the environment.
* **Increased Autonomy**: RAGs allow AI systems to operate independently, making decisions and taking actions without human intervention.
* **Enhanced Flexibility**: RAGs enable AI systems to adapt to changing situations and environments.

---

**Benefits of Agentic AI RAGs**
=====================================

### Improved Decision-Making

Agentic AI RAGs (Reasoning and Argumentation Graphs) can significantly enhance decision-making capabilities by providing a structured and transparent approach to evaluating complex information. These AI systems can analyze vast amounts of data, identify patterns, and make informed recommendations, reducing the likelihood of human error.

### Enhanced Problem-Solving

Agentic AI RAGs facilitate effective problem-solving by breaking down complex issues into manageable components and identifying potential solutions. This process enables the AI system to weigh the pros and cons of each option, leading to more efficient and effective decision-making.

### Increased Efficiency

The use of agentic AI RAGs can automate routine tasks, freeing up human resources for more strategic and creative work. By streamlining processes and reducing the need for manual data analysis, these AI systems can significantly improve efficiency across various industries.

### Improved Collaboration

Agentic AI RAGs can facilitate collaboration among stakeholders by providing a shared understanding of the problem or opportunity. This enables teams to work together more effectively, sharing insights and expertise to drive better outcomes.

### Adaptable and Scalable

Agentic AI RAGs can adapt to changing circumstances and scale to meet the needs of complex organizations. This flexibility makes them an attractive solution for industries that require rapid response times and continuous innovation.

### Reduced Bias and Error

Agentic AI RAGs can help mitigate the impact of human bias and error by providing a data-driven approach to decision-making. By analyzing objective data and avoiding emotional or personal influences, these AI systems can deliver more accurate and reliable results.

---

### Applications of Agentic AI RAGs

#### Real-world use cases of agentic AI RAGs in areas like customer service, healthcare, finance, and more

#### Customer Service

Agentic AI RAGs can revolutionize customer service by providing 24/7 support, reducing wait times, and increasing customer satisfaction. Some potential applications include:

* **Chatbots**: AI-powered chatbots can engage with customers, answer their queries, and provide solutions to their problems. They can be integrated into various platforms like websites, mobile apps, and messaging platforms.
* **Virtual Assistants**: Virtual assistants like Amazon's Alexa and Google Assistant can use agentic AI RAGs to provide personalized support to customers, book appointments, and make purchases.
* **Predictive Maintenance**: Agentic AI RAGs can analyze customer interactions and predict potential issues, enabling proactive customer support and reducing the likelihood of complaints.

#### Healthcare

Agentic AI RAGs can transform the healthcare industry by improving patient outcomes, reducing costs, and enhancing the overall patient experience. Some potential applications include:

* **Personalized Medicine**: Agentic AI RAGs can analyze patient data, medical history, and genetic profiles to provide personalized treatment plans and recommendations.
* **Virtual Nursing Assistants**: Virtual nursing assistants can use agentic AI RAGs to monitor patient vital signs, detect potential health issues, and provide guidance to patients on medication adherence and self-care.
* **Clinical Decision Support**: Agentic AI RAGs can analyze medical data and provide healthcare professionals with real-time insights and recommendations to improve diagnosis and treatment outcomes.

#### Finance

Agentic AI RAGs can transform the finance industry by improving risk management, reducing costs, and enhancing customer experience. Some potential applications include:

* **Risk Management**: Agentic AI RAGs can analyze financial data, detect potential risks, and provide insights to financial institutions to minimize losses.
* **Virtual Assistants**: Virtual assistants can use agentic AI RAGs to provide customers with personalized financial advice, help them manage their accounts, and make informed investment decisions.
* **Automated Trading**: Agentic AI RAGs can analyze market trends, detect potential opportunities, and execute trades automatically, reducing the need for human intervention.

#### Education

Agentic AI RAGs can transform the education industry by improving student outcomes, reducing costs, and enhancing the overall learning experience. Some potential applications include:

* **Personalized Learning**: Agentic AI RAGs can analyze student data, learning styles, and preferences to provide personalized learning plans and recommendations.
* **Virtual Teaching Assistants**: Virtual teaching assistants can use agentic AI RAGs to provide students with real-time support, answer their questions, and provide feedback on their work.
* **Intelligent Tutoring Systems**: Agentic AI RAGs can analyze student performance, identify knowledge gaps, and provide targeted interventions to improve student outcomes.

These are just a few examples of the many potential applications of agentic AI RAGs in various industries. As the technology continues to evolve, we can expect to see even more innovative and impactful uses of agentic AI RAGs in the future.

---

**Challenges and Limitations of Agentic AI RAGs**

### 1. Potential Risks of Agentic AI RAGs

Agentic AI RAGs have the potential to pose significant risks to individuals, organizations, and society as a whole. Some potential risks include:

- **Loss of Control**: Agentic AI RAGs may become difficult to control, as they learn and adapt to their environment, making it challenging to predict their behavior.
- **Bias and Discrimination**: AI RAGs may perpetuate existing biases and discrimination, leading to unfair outcomes and exacerbating social inequalities.
- **Cybersecurity Threats**: Agentic AI RAGs may be vulnerable to cyber attacks, which could compromise sensitive information and disrupt critical systems.
- **Dependence and Addiction**: The use of agentic AI RAGs may lead to dependence and addiction, as individuals become increasingly reliant on these systems for decision-making and problem-solving.

### 2. Biases in Agentic AI RAGs

Agentic AI RAGs can perpetuate biases and discrimination in several ways:

- **Data Bias**: AI RAGs may learn from biased data, which can result in biased decision-making and outcomes.
- **Algorithmic Bias**: The algorithms used in AI RAGs may contain biases, leading to unfair treatment of certain groups or individuals.
- **Human Bias**: The design and development of AI RAGs may be influenced by human biases, which can be reflected in the final product.

### 3. Limitations of Agentic AI RAGs

Agentic AI RAGs have several limitations that must be addressed:

- **Lack of Contextual Understanding**: AI RAGs may not fully understand the context in which they are operating, leading to misinterpretation and poor decision-making.
- **Limited Knowledge**: AI RAGs may not have access to the same level of knowledge and expertise as human experts, limiting their ability to make informed decisions.
- **Vulnerability to Adversarial Attacks**: AI RAGs may be vulnerable to adversarial attacks, which can compromise their performance and decision-making abilities.

### Addressing the Challenges and Limitations of Agentic AI RAGs

To address the challenges and limitations of agentic AI RAGs, the following steps can be taken:

- **Diverse and Inclusive Development Teams**: Ensure that development teams are diverse and inclusive to reduce the risk of bias and promote more equitable outcomes.
- **Regular Auditing and Testing**: Regularly audit and test AI RAGs to identify and mitigate biases and errors.
- **Transparency and Explainability**: Ensure that AI RAGs are transparent and explainable, providing clear insights into their decision-making processes and outcomes.
- **Human Oversight and Review**: Implement human oversight and review processes to ensure that AI RAGs are operating within acceptable parameters and making fair and informed decisions.

---

### Future of Agentic AI RAGs

#### Emerging Trends

As the field of artificial intelligence continues to evolve, agentic AI RAGs (Robust Aggregates of Goals) are becoming increasingly prominent. One emerging trend in agentic AI RAGs is the development of more sophisticated goal-oriented architectures. These architectures enable AI systems to learn from experience, adapt to new situations, and make decisions that align with their goals.

Another trend is the integration of cognitive architectures with deep learning techniques. This integration enables agentic AI RAGs to leverage the strengths of both symbolic and connectionist AI, resulting in more robust and flexible goal-oriented systems.

#### Advancements

Recent advancements in agentic AI RAGs have focused on improving their ability to:

* **Learn from Experience**: Agentic AI RAGs can now learn from experience and adapt to new situations, enabling them to improve their performance over time.
* **Reason about Objectives**: Agentic AI RAGs can reason about their objectives and make decisions that align with their goals, even in the presence of uncertainty or conflicting information.
* **Understand Human Behavior**: Agentic AI RAGs can now understand human behavior and preferences, enabling them to interact more effectively with humans.
* **Collaborate with Humans**: Agentic AI RAGs can collaborate with humans to achieve common goals, such as completing complex tasks or solving complex problems.

#### Potential Future Applications

The potential future applications of agentic AI RAGs are vast and varied. Some possible applications include:

* **Autonomous Systems**: Agentic AI RAGs can be used to develop autonomous systems that can learn, adapt, and make decisions in complex environments.
* **Virtual Assistants**: Agentic AI RAGs can be used to develop virtual assistants that can learn from experience and adapt to new situations.
* **Robotics**: Agentic AI RAGs can be used to develop robots that can learn from experience and adapt to new situations.
* **Healthcare**: Agentic AI RAGs can be used to develop personalized healthcare systems that can learn from experience and adapt to new situations.

#### Challenges and Limitations

While agentic AI RAGs hold great promise, there are several challenges and limitations that need to be addressed. Some of these challenges include:

* **Safety and Control**: Agentic AI RAGs need to be designed with safety and control mechanisms to prevent them from causing harm or malfunctioning.
* **Transparency and Explainability**: Agentic AI RAGs need to be designed with transparency and explainability mechanisms to ensure that their decisions and actions are understandable and interpretable.
* **Scalability**: Agentic AI RAGs need to be designed to scale to complex environments and tasks, while maintaining their performance and reliability.

By addressing these challenges and limitations, researchers and developers can unlock the full potential of agentic AI RAGs and create more sophisticated, adaptable, and effective goal-oriented systems.